In [33]:
!pip install Pillow
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from PIL import Image, ImageCms
import numpy as np
import os
import random
import tensorflow as tf

In [34]:
photos_path = './faces/Train/'
test_path = './faces/Test/'
# photos_path = './unsplash_photos/'
RGB = 'RGB'
LAB = 'LAB'
train_split = 0.8
random_seed = 42
resolution = (256, 256)


# Converter for Lab colourspace
srgb_p = ImageCms.createProfile("sRGB")
lab_p  = ImageCms.createProfile("LAB")

rgb2lab = ImageCms.buildTransformFromOpenProfiles(srgb_p, lab_p, "RGB", "LAB")
lab2rgb = ImageCms.buildTransformFromOpenProfiles(lab_p, srgb_p, "LAB", "RGB")

In [35]:
photos = []
for photo in os.listdir(photos_path):
    rgb_photo = Image.open(os.path.join(photos_path, photo)).resize(resolution).convert(RGB)
    lab_photo =  ImageCms.applyTransform(rgb_photo, rgb2lab)
    lab_array = np.array(lab_photo)
    photos.append(lab_array)

In [36]:
photos_np = np.array(photos, dtype= float)
photos_np /= 255.0

split_len = int(len(photos_np) * train_split)

photos_train = photos_np[:split_len]
photos_val = photos_np[split_len:]

In [37]:
photos_np.shape

(10, 256, 256, 3)

In [38]:
neurons_set_1 = [64, 128, 256]
neurons_set_2 = [512, 256, 128]
upsampling_neurons = [64, 32]
filter_size = (3,3)
upsampling_filter_size = (2,2)
activation_hidden = 'relu'
activation_output = 'tanh'
strides = 2

epochs = 10
steps_per_epoch = 100

# create model
model = Sequential()

# Input layer
model.add(InputLayer(input_shape=(256, 256, 1)))

# Hidden layer set 1
for neuron in neurons_set_1:
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same', strides=strides))
    
# Hidden layer set 2
for neuron in neurons_set_2:
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))

# Upsampling Hidden layer
for neuron in upsampling_neurons:
    model.add(UpSampling2D(upsampling_filter_size))
    model.add(Conv2D(neuron, filter_size, activation=activation_hidden, padding='same'))

# prepare output layer
model.add(Conv2D(2, (3,3), activation=activation_output, padding='same'))
model.add(UpSampling2D(upsampling_filter_size))

model.compile(optimizer = 'rmsprop', loss='mse')


In [39]:
# Image transformer
datagen = ImageDataGenerator(
    shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

# Generate training data
batch_size = 10
def batch_generator(batch_size):
    for photos_batch in datagen.flow(photos_train, batch_size=batch_size):
        X_batch = photos_np[:,:,:,0]
        y_batch = photos_np[:,:,:,1:]
        y_batch -= 128
        y_batch /= 128
        yield (X_batch.reshape(X_batch.shape + (1,)), y_batch)

# Train model
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(batch_generator(batch_size), callbacks=[tensorboard], epochs=epochs, steps_per_epoch=steps_per_epoch)


Epoch 1/10
100/100 [==============================] - 17s 170ms/step - loss: 163.8719
Epoch 2/10
100/100 [==============================] - 17s 172ms/step - loss: 6.2000e-05
Epoch 3/10
100/100 [==============================] - 17s 169ms/step - loss: 6.2000e-05
Epoch 4/10
100/100 [==============================] - 17s 172ms/step - loss: 6.2000e-05
Epoch 5/10
100/100 [==============================] - 23s 228ms/step - loss: 6.2000e-05
Epoch 6/10
100/100 [==============================] - 20s 203ms/step - loss: 6.2000e-05
Epoch 7/10
100/100 [==============================] - 26s 262ms/step - loss: 6.2000e-05
Epoch 8/10
100/100 [==============================] - 22s 220ms/step - loss: 6.2000e-05
Epoch 9/10
100/100 [==============================] - 20s 202ms/step - loss: 6.2000e-05
Epoch 10/10
100/100 [==============================] - 23s 229ms/step - loss: 6.2000e-050s - loss: 6.2000


In [40]:
X_val = photos_val[:,:,:,0]
X_val = X_val.reshape(X_val.shape + (1,))

y_val = photos_val[:,:,:,1:]
y_val /= 128

print(model.evaluate(X_val, y_val, batch_size=batch_size))

2/2 [==============================] - 0s 104ms/step
6.200001371325925e-05


In [41]:
testing_set = []
for photo in os.listdir(test_path):
    rgb_photo = Image.open(os.path.join(test_path, photo)).resize(resolution).convert(RGB)
    lab_photo =  ImageCms.applyTransform(rgb_photo, rgb2lab)
    lab_array = np.array(lab_photo)
    testing_set.append(lab_array)

testing_set = np.array(testing_set, dtype = float)
testing_set /= 255.0

test_photos = testing_set[:,:,:,0]
test_photos = test_photos.reshape(test_photos.shape + (1,))

output = model.predict(test_photos)
output *= 128
output += 128
output = output.astype(int)
output[0]

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]])

In [42]:
output_path = f"Result/face_{epochs}_{steps_per_epoch}/"

# Check whether the specified path exists or not
exists = os.path.exists(output_path)
if not exists:
   # Create a new directory because it does not exist
   os.makedirs(output_path)
   print("The new directory is created!")
   
for i in range(len(output)):
    canvas = np.zeros((256, 256, 3))
    bw_part = testing_set[i][:,:,0]
    bw_part *= 255
    bw_part = bw_part.astype(int)
    canvas[:,:,0] = bw_part
    canvas[:,:,1:] = output[i]
    lab_image = Image.fromarray(canvas, mode="LAB")
    rgb_image = ImageCms.applyTransform(lab_image, lab2rgb)
    rgb_image.save(output_path+f"output_{i}.jpeg")


The new directory is created!
